Dysgu API for python3
==================

This notebook gives an overview of the dysgu-api for python.


Calling structural variants
------------------------------

Dysgu relies on the `pysam` library to provide access to alignment files and the reference genome. To call SVs, first initialize a `DysguSV` class with an indexed reference genome and alignment file using pysam. Here we will use the HG002 samples which is ~40x coverage:

In [1]:
import time
import pysam
from dysgu import DysguSV

bam = pysam.AlignmentFile('HG002.bam', 'rb')
genome = pysam.FastaFile('ucsc.hg19.fasta')

dysgu = DysguSV(genome, bam)

Calling SVs is performed by passing an iterable of reads to dysgu. For example to call SVs over the first 1 Mb of chr1:

In [2]:
t0 = time.time()

df = dysgu("chr1:1-1000000")

print(f"Time: {time.time() - t0:.3f} seconds")

# Dysgu can also be called with an iterable, for example:
f_iter = bam.fetch("chr1", 10_000, 20_000)
df2 = dysgu(f_iter)

Time: 1.890 seconds


Dysgu will return a pandas DataFrame with the resulting calls, or `None` if no SVs were identified:

In [3]:
df.head(10)

,chrA,posA,event_id,ref_seq,variant_seq,filter,sample,svtype,posB,chrB,...,inner_cn,outer_cn,compress,ref_rep,jitter,contigA,right_ins_seq,left_ins_seq,partners,prob
0,chr1,54720,76,C,TTTCTTTCTTTTTTTCTTTTTTTTTTTCTTTCTTTCTTTCTTTCTT...,PASS,sample,INS,54720,chr1,...,2.25,0.000,31.219999,0.000000,0.0,ccttctattttttttttctttctttctttttttctttttttttttc...,None,None,None,0.626
1,chr1,66321,81,A,<INS>,lowProb,sample,INS,66321,chr1,...,16.50,1.500,19.750000,0.000000,0.0,aatatatatattattatataatataatatatattATATAATATATA...,None,aatatatatattattatataatataatatatatt,None,0.427
2,chr1,66452,85,A,<INS>,PASS,sample,INS,66452,chr1,...,17.50,0.750,19.020000,0.000000,0.0,ATATTATTATATAATATAATATATATTATATAATATATTTTATTAT...,atatatattatataatatattttattatataaatatatattatattat,None,None,0.758
3,chr1,724219,187,A,TGGAATGAAGTGGAATGGAGTGGAATTGATTGGTGCGGAGCAGAAA...,lowProb,sample,INS,724219,chr1,...,52.25,21.750,36.560001,0.000000,0.0,tggaatgcaatgaaacagaatggaatggattggatttgtatggact...,None,None,None,0.339
4,chr1,724350,199,T,<DEL>,lowProb,sample,DEL,724468,chr1,...,7.00,12.375,36.139999,0.320301,0.0,,None,None,None,0.076
5,chr1,724605,231,T,AAGGAATTTATTCGAATGGAATAGAATGGAAAGGA,PASS,sample,INS,724708,chr1,...,53.50,22.000,37.990002,0.000000,0.0,ccGAATGGAATGGAATGTAATGGAACGGAATGGAATGGAACAGAAT...,tgaatggaaatgaaagtaata,None,None,0.518
6,chr1,725033,196,T,<INS>,lowProb,sample,INS,726715,chr1,...,22.50,7.000,37.020000,0.000000,0.0,,None,atgtacttgaatggagtagaatggaatggaatgtaaaggaaatgaa...,None,0.116
7,chr1,725302,318,G,<INS>,lowProb,sample,INS,725361,chr1,...,20.25,21.750,33.680000,0.000000,0.0,tgaaatggaatggaatggaatagattccaaagaatggaatggactc...,None,tgaaatggaatggaatggaatagattccaaagaatggaatggactc...,None,0.286
8,chr1,725717,351,G,<INS>,lowProb,sample,INS,725763,chr1,...,46.00,13.500,35.900002,0.000000,0.0,ggaagggaatggagtgtacaagaatggaatgggatggcaaaaaatg...,None,ggaagggaatggagtgtacaagaatggaatgggatggcaaaaaatg...,None,0.211
9,chr1,726095,358,T,<DEL>,lowProb,sample,DEL,726125,chr1,...,10.00,7.875,41.450001,0.305509,0.0,GGAATAGAATAGAATGGACTGGAATGTAATGAGTTTGGAATGGAAT...,None,None,None,0.121


In [4]:
# display the number of calls with a PASS in the filter column:
print(df['filter'].value_counts())

# display the counts of SV types with a PASS
print(df.query(" filter == 'PASS' ")['svtype'].value_counts());

lowProb    15
PASS       12
Name: filter, dtype: int64
INS    9
DEL    3
Name: svtype, dtype: int64


Notes
-------
- Dysgu expects the input alignment file to be in sorted order

- There is currently some overhead associated with preparing a new chromosome for SV calling, for this reason it will be inefficient to repeatedly call multiple small regions. It is far more efficient to iterate over all target regions in one go. Dysgu provides a few tools for manipulating bed files to support this pattern:

In [5]:
from dysgu import load_bed, merge_intervals

bed = load_bed('example.bed')

# merge and sort intervals
bed = merge_intervals(bed, srt=True)
print(bed)

t0 = time.time()

df = dysgu.call_bed_regions(bed)

print(time.time() - t0)
df['filter'].value_counts()

[('chr1', 10000, 21000), ('chr1', 25000, 30000), ('chr1', 35000, 36000), ('chr1', 1500000, 16000000)]
11.701174020767212


lowProb    189
PASS       132
Name: filter, dtype: int64

Options
---------

The default options for dysgu are similar to the CLI application. These can be inspected using:

In [6]:
import pprint
from dysgu import dysgu_default_args

pprint.pprint(dysgu_default_args())

{'add_gt': True,
 'add_kind': True,
 'all_sites': False,
 'buffer_size': 10000,
 'clean': False,
 'clip_length': 15,
 'contigs': True,
 'diploid': True,
 'dist_norm': 100,
 'drop_gaps': True,
 'ibam': None,
 'keep_small': False,
 'low_mem': False,
 'max_cov': 200,
 'max_tlen': 1000,
 'merge_dist': None,
 'merge_within': True,
 'metrics': True,
 'min_size': 30,
 'min_support': 3,
 'mode': 'pe',
 'model': None,
 'mq': 1,
 'overwrite': True,
 'paired': True,
 'parse_probs': False,
 'pfix': 'dysgu_reads',
 'pl': 'pe',
 'procs': 1,
 'reference': None,
 'regions': None,
 'regions_mm_only': False,
 'regions_only': False,
 'remap': True,
 'sites': None,
 'sites_pass_only': True,
 'sites_prob': 0.6,
 'spd': 0.3,
 'sv_aligns': None,
 'template_size': '',
 'thresholds': {'DEL': 0.45,
                'DUP': 0.45,
                'INS': 0.45,
                'INV': 0.45,
                'TRA': 0.45},
 'trust_ins_len': True,
 'verbosity': 2,
 'working_directory': 'tempfile',
 'z_breadth': 2,
 'z_dep

Options are specified as key word arguments during initialization. For example, to change the `min-support` option:

In [7]:
dysgu = DysguSV(genome, bam, min_support=5)
df = dysgu.call_bed_regions(bed)
df['filter'].value_counts()

PASS       87
lowProb    87
Name: filter, dtype: int64

In [8]:
# Or alternatively:
dysgu.set_option("min_support", 10)

dysgu.call_bed_regions(bed)['filter'].value_counts()

PASS       46
lowProb    34
Name: filter, dtype: int64

In [9]:
# Or provide a mapping of arguments:
dysgu.set_option({"min_support": 10, "mq": 20, "min_size": 100})

dysgu.call_bed_regions(bed)['filter'].value_counts()

PASS       26
lowProb    14
Name: filter, dtype: int64

Saving and loading a VCF
------------------------------

`to_vcf` and `load_vcf` functions can be used as follows:

In [10]:
from dysgu import load_dysgu_vcf
import pandas as pd


dysgu = DysguSV(genome, bam)

df = dysgu("chr1:1000000-2000000")

# Note a sample name in needed for the vcf file, this can be set as an attribute,
# or during construction i.e. DysguSV(genome, bam, 'sample1')
dysgu.sample_name = "sample1"

with open("output.vcf", "w") as out:
    dysgu.to_vcf(df, out)

df2 = load_dysgu_vcf("output.vcf")


# just to check dataframes are the same. Note float values are rounded when saving to vcf
pd.testing.assert_frame_equal(df[["chrA", "posA", "event_id"]],
                              df2[["chrA", "posA", "event_id"]])

Switching machine learning models
------------------------------------------

To change the ML model, first configure dysgu with the desired options, then use the `apply_model` function:

In [11]:
dysgu = DysguSV(genome, bam)
df = dysgu("chr1:1000000-2000000")

# try the non-diploid model
dysgu.set_option("diploid", False)
df_non_diploid = dysgu.apply_model(df)
print(df_non_diploid['filter'].value_counts())

# try the diploid model
dysgu.set_option("diploid", True)
df_diploid = dysgu.apply_model(df)
print(df_diploid['filter'].value_counts())

# try the no-contigs model
dysgu.set_option("contigs", False)
df_no_conts = dysgu.apply_model(df)
print(df_no_conts['filter'].value_counts())

# try the pacbio model (why not)
dysgu.set_option({"contigs": False, "pl": "pacbio"})
df_no_conts = dysgu.apply_model(df)
print(df_no_conts['filter'].value_counts())

lowProb    21
PASS       16
Name: filter, dtype: int64
lowProb    24
PASS       13
Name: filter, dtype: int64
lowProb    23
PASS       14
Name: filter, dtype: int64
PASS       19
lowProb    18
Name: filter, dtype: int64


Merging SVs
---------------

Merging is supported via the `merge_dysgu_df` function:

In [12]:
from dysgu import merge_dysgu_df


samp1 = DysguSV(genome, pysam.AlignmentFile('HG002.bam', 'rb'), 'HG002')
samp2 = DysguSV(genome, pysam.AlignmentFile('NA12878.bwa.bam', 'rb'), 'NA12878')

r = "chr1:8705980-8706344"

df_samp1 = samp1(r)
df_samp2 = samp2(r)


In [13]:
df_samp1

,chrA,posA,event_id,ref_seq,variant_seq,filter,sample,svtype,posB,chrB,...,inner_cn,outer_cn,compress,ref_rep,jitter,contigA,right_ins_seq,left_ins_seq,partners,prob
0,chr1,8706162,1,T,<DEL>,PASS,HG002,DEL,8706208,chr1,...,1.7,0.0,47.43,0.183956,0.0,ACTATAATGTGACTGAAAATAAGAACTACATTTTCCCAACTGTTCC...,None,None,None,0.887


In [14]:
df_samp2

,chrA,posA,event_id,ref_seq,variant_seq,filter,sample,svtype,posB,chrB,...,inner_cn,outer_cn,compress,ref_rep,jitter,contigA,right_ins_seq,left_ins_seq,partners,prob
0,chr1,8706162,1,T,<DEL>,PASS,NA12878,DEL,8706208,chr1,...,0.833333,0.0,47.939999,0.183956,0.0,CACTATAATGTGACTGAAAATAAGAACTACATTTTCCCAACTGTTC...,None,None,None,0.795


In [16]:
merged = merge_dysgu_df(df_samp1, df_samp2)
merged

,chrA,posA,event_id,ref_seq,variant_seq,filter,sample,svtype,posB,chrB,...,outer_cn,compress,ref_rep,jitter,contigA,contigA,right_ins_seq,left_ins_seq,partners,prob
0,chr1,8706162,0,T,<DEL>,PASS,HG002,DEL,8706208,chr1,...,0.0,47.43,0.183956,0.0,ACTATAATGTGACTGAAAATAAGAACTACATTTTCCCAACTGTTCC...,ACTATAATGTGACTGAAAATAAGAACTACATTTTCCCAACTGTTCC...,None,None,"[(1, 1)]",0.887


Following merging, the `partners` column will contain a list of the merged variants. Each item merged is represented as a tuple with the format `(df_index, event_id)`. In the example above df_samp1 would have `df_index = 0` and df_samp2 would have `df_index = 1`. 

In the partners column, the list `[(1, 1)]` indicates that the variant was merged with a variant from df_index=1 with event_id=1, hence (1, 1).

Note, the `sample` column is not modified during merging, so this is left to the user to manipulate if needs be